###Trick to work with current code in repo
This add the location of the fc files in this repo to the system paths so it can be imported as normal

In [1]:
import os, sys
lib_path = os.path.abspath(os.path.join('..'))
sys.path.append(lib_path)

In [2]:
import xlrd
import collections
def import_rows(workbook_name,
                worksheet_name):
    '''
    Access an excel doument and imports data from a specific spreadsheet.
    Returns a list of dictionaries, which contain key value pairs of the data
    *  The spreadsheet should contain a header row with the names of each key.
    *  Each subsequent row then contains the values contained in a single dictionary.
    *  Header row keys are parsed from A1 until a column without a entry is reached.
    *  Dictionaries are parsed until a row is reached which as no values for any key.
        This means that empty values for a key are acceptable for a row
    '''
    rows_data = []
    # Load workbook and sheet
    workbook = xlrd.open_workbook(workbook_name)
    if 'cells' not in workbook.sheet_names():
        raise IOError('No sheet named cells! Name the worksheet used to import data cells please')
    worksheet = workbook.sheet_by_name(worksheet_name)
    for r in range(worksheet.nrows):
        row = worksheet.row(r)
        #Import headers
        if r==0:
            headers = [header.value for header in row]
        #Import row
        else:
            row_data = collections.OrderedDict()
            for c, cell in enumerate(row):
                value = cell.value
                if value != None and isinstance(value, basestring) and len(value) > 0 and value[0] == '=':
                    raise ImportError("Error: Ran into an excel formula, use plain text only. ("+str(r)+', '+str(c)+')')
                if headers[c]!= '':
                    row_data[headers[c]] = value                
            #If row was empty breakt the for loop
            if sum([v != None for k, v in row_data.iteritems()]) == 0:
                print [v is None for k, v in row_data.iteritems()]
                break;
            else:
                rows_data.append(row_data)
    return rows_data

import xlwt
def export_workbook(workbook_name, worksheet_data):
    '''
    Writes a list of lists to a excel file
    Overwrite cells but not worksheets
    The first dimension is associated with the column, the second
    dimension is associated with the row
    '''
    #Create Workbook
    workbook = xlwt.Workbook()
    
    for name, data in worksheet_data.iteritems():
        sheet = workbook.add_sheet(name)
        for c, column_vector in enumerate(data):
            for r, value in enumerate(column_vector):
                sheet.write(r, c, value)    
    try:
        workbook.save(workbook_name)
    except IOError:
        raise IOError("Excel document must be closed! Please save and close it and run this script again!")

In [3]:
import matplotlib.pyplot as plt
import fc.plot
import gc
def gate_plot(ungated_points, gated_points, contour, title='', filename=None):
    '''
    Plots both the FSC SSC scatterplot with gate, and the overlay of channel one gated and ungated histograms
    Used to confirm automated gating was approriate for each sample
    '''
    fig = plt.figure(figsize=(12,9))
    density_ax = fig.add_subplot(2,1,1)
    
    fc.plot.density2d(ungated_points, gate = contour, sigma=2.5, ax = density_ax, colorbar=False,xlabel='FSC-H',ylabel='SSC-H')
    density_ax.set_aspect('auto')
    plt.title(title)
    
    flour_ax = fig.add_subplot(2,1,2)
    fc.plot.hist1d(ungated_points[:,2],ax = flour_ax,edge_color=(0.5, 0.85, 0.3), face_color=(0.8, 0.95, 0.7))
    fc.plot.hist1d(gated_points[:,2],ax = flour_ax, edge_color=(0.2, 0.7, 0), face_color=(0.6, 0.9, 0.4),xlabel='FL1-H')
    plt.tight_layout()
    
    if filename != None:
        filename = plot_gated_folder + '/gated_%03d.png'%(i+1)
        plt.savefig(filename, bbox_inches='tight')
    
    plt.close()
    fig.clf()
    gc.collect();

In [18]:
import numpy
def generate_stats(samples):
    '''
    Takes a set of samples, generate stats for them
    
    More stats can be added in the future
    '''
    def function_specifier(fn, channel=0, attribute=''):
        def fn_wrapper(data):
            return fn(getattr(sample, attribute, sample), channel) #pass any arguments to fn()
        return fn_wrapper

    def counts(data, channel):
        return data[:,channel].size
    
    def mean(data, channel):
        return numpy.mean(data[:,channel])

    def mode(data, channel):
        return numpy.argmax(numpy.bincount(data[:,channel].astype('int32')))

    def std(data, channel):
        return numpy.std(data[:,channel])

    def CV(data, channel):
        return numpy.std(data[:,channel])/numpy.mean(data[:,channel])
    
    def median(data, channel):
        return numpy.median(data[:,channel])

    def iqr(data, channel):
        q75, q25 = numpy.percentile(data[:,channel], [75 ,25])
        return q75 - q25
    
    def RCV(data, channel):
        q75, q25 = numpy.percentile(data[:,channel], [75 ,25])
        return numpy.median(data[:,channel])/(q75 - q25)
    
    stat_functions = []

    data_attribute = 'data_transformed'
    
    stat_functions.append(('Ungated Counts',function_specifier(counts)))
    stat_functions.append(('Gated Counts',function_specifier(counts, attribute = data_attribute)))
    
    for channel in ['FL1-H', 'FL2-H', 'FL3-H']:
        stat_functions.append((channel+' Mean',function_specifier(mean, channel, data_attribute)))
        stat_functions.append((channel+' Mode',function_specifier(mode, channel, data_attribute)))
        stat_functions.append((channel+' Std',function_specifier(std, channel, data_attribute)))
        stat_functions.append((channel+' CV',function_specifier(CV, channel, data_attribute)))
        stat_functions.append((channel+' Median',function_specifier(median, channel, data_attribute)))
        stat_functions.append((channel+' IQR',function_specifier(iqr, channel, data_attribute)))
        stat_functions.append((channel+' RCV',function_specifier(RCV, channel, data_attribute)))

    output = []
    
    for header in samples[0].metadata.keys():
        output.append([header])
    for name, stat_function in stat_functions:
        output.append([name])
        
    for sample in samples:
        i = 0
        # Add meta data to output array
        for header, value in sample.metadata.iteritems():
            if value == None:
                value = ''
            output[i].append(value)
            i+=1
        for name, stat_function in stat_functions:
            value = stat_function(sample.data_transformed)
            output[i].append(value)
            i+=1
    return output

In [19]:
import fc.gate
def auto_gate(sample, gate_fraction=.2, sigma=10):
    '''
    * Gates out first and last channel of all 5 channels
    * Gates out first 250 events and last 100 events of run
    * Performs a density gate on remaining data
    * Transforms data to an exponential form
    '''
    
    gate_fraction = float(sample.metadata['Gate Fraction']) if 'Gate Fraction' in sample.metadata and sample.metadata['Gate Fraction'] != '' else gate_fraction
    sigma = float(sample.metadata['Sigma']) if 'Sigma' in sample.metadata and sample.metadata['Sigma'] != '' else sigma
    
    high_low_gate = fc.gate.high_low(sample, high=(2**10)-1, low=0)
    start_stop_gate = fc.gate.start_stop(sample, num_start=250, num_stop=100)
    data_coarse = sample[high_low_gate*start_stop_gate]
    
    density2d_gate, density2d_contour = fc.gate.density2d(data_coarse,gate_fraction = gate_fraction,sigma = sigma)
    data_fine = data_coarse[density2d_gate]
    
    return data_fine, data_coarse, density2d_contour

In [20]:
import fc.transform
def auto_transform(data, metadata, cached_beads):
    
    beads_file_path = metadata['Beads File Path'] if 'Beads File Path' in metadata and metadata['Beads File Path'] != '' else None
    beads_peaks = [float(x) for x in metadata['Beads Peaks'].split(',')] if 'Beads Peaks' in metadata and metadata['Beads Peaks'] != '' else None
    
    #If they just include one parameter
    if beads_file_path != None and beads_peaks == None or beads_file_path == None and beads_peaks != None:
        raise ImportError('You must provide both the Beads File Path and the Beads Peaks (see: '+metadata['File Path']+')')
    
    #If beads are not provided just exponentiate
    if beads_file_path == None and beads_peaks == None:
        return fc.transform.exponentiate(data)
    
    #If file/peaks have already been cached use them, otherwise compute new ones
    if beads_file_path in cached_beads.keys():
        mefl_transform = cached_beads[beads_file_path]
    else:
        bead_data = fc.io.TaborLabFCSData(beads_file_path)
        # common gating/trimming
        ss_m = fc.gate.start_stop(bead_data[:,[0,1]])
        hl_m = fc.gate.high_low(bead_data[:,[0,1]])
        cm = ss_m & hl_m
        trimmed_data = bead_data[cm,:]

        # density gate
        dm,cntr = fc.gate.density2d(data=trimmed_data, gate_fraction=0.4, sigma=10)
        gated_data = trimmed_data[dm]

        # mef
        mefl = [None, 792.,2079.,6588.,16471.,47497.,137049.,271647.]
        peaks_mef = numpy.array(mefl)
        o = fc.transform.get_mef_standard_curve(gated_data, peaks_mef, 
           cluster_channels = ['FL1-H', 'FL2-H', 'FL3-H'],
           mef_channels = 'FL1-H', verbose = False,);
        mefl_transform = o[0]
        beads[beads_file_path] = mefl_transform
    return mefl_transform(data)

In [24]:
import fc.gate, fc.transform, fc.io, os
from Tkinter import Tk
from tkFileDialog import askopenfilename



# Import bead metadata
# Load bead data
# Analysis bead data
# Graph bead data
# Import cell metadata

#Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
#import_path = askopenfilename() # show an "Open" dialog box and return the path to the selected file
import_path = "input_form.xlsx"
directory = os.path.dirname(os.path.realpath(import_path))
print "Loading files..."
# Load Files
samples = []
for row in import_rows(import_path,"cells"):
    sample = fc.io.TaborLabFCSData(row['File Path'])
    sample.metadata = row
    samples.append(sample)
    
# Generate Gates
print "Generating gate for file:",
beads_cache = {}
for i, sample in enumerate(samples):
    print str(i+1)+', ',
    sample.data_fine, sample.data_coarse, sample.density2d_contour = auto_gate(sample)
    transformed_data = auto_transform(sample, sample.metadata, beads_cache)
    sample.data_transformed = transformed_data
print ''
    
# Plot graphs of gating
print "Generating plots for file:",
plot_gated_folder = os.path.join(directory,'plot_gated')
if not os.path.exists(plot_gated_folder):
    os.makedirs(plot_gated_folder)
for i, sample in enumerate(samples): 
    print str(i+1)+', ',
    gate_plot(sample.data_coarse, sample.data_fine, sample.density2d_contour, title = sample.metadata['File Path'], filename = os.path.join(plot_gated_folder,'gated_%03d.png'%(i+1)))
print ''

# Generate stats
print 'Generating stats...'
output = generate_stats(samples)
export_workbook(os.path.join(directory,'mefled_data.xls'),{'stats2':output})
print 'All done!'  

Loading files...
Generating gate for file: 1,  2,  3,  4,  5,  6,  7,  
Generating plots for file: 1,  2,  3,  4,  5,  6,  7,  
Generating stats...
All done!
